---
#### final data 로 ml 학습 


In [4]:
import pandas as pd 
data =pd.read_csv('./Data/final.csv')

In [8]:
data.columns

Index(['Unnamed: 0', '시설아이디', 'weeks', '조사일자', '표본번호', '관부직경', '엽병장', '엽수',
       '엽장', '엽폭', '개화수', '초장', '최종화방차수', '화방 꽃수(소화수)', 'supplyEC', 'supplyPH',
       'innerCO2', 'innerHum', 'innerTemp', 'innerSolar', 'CO2'],
      dtype='object')

In [20]:
input_cols =['weeks','supplyEC', 'supplyPH',
       'innerCO2', 'innerHum', 'innerTemp', 'innerSolar', 'CO2']
output_cols = ['관부직경', '엽병장', '엽수',
       '엽장', '엽폭', '개화수', '초장', '최종화방차수', '화방 꽃수(소화수)']
training_table = data.loc[:,input_cols]
target_table= data.loc[:,output_cols]

In [2]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor

In [21]:
train_input, test_input, train_target, test_target = \
    train_test_split(training_table,
                    target_table, 
                    test_size=0.2,
                    random_state=42)
    
## KNN regression model 
knn_regressor = KNeighborsRegressor(n_neighbors=3)
## Multi Output Setting
multi_output_regressor = MultiOutputRegressor(knn_regressor)
multi_output_regressor.fit(train_input, train_target)

score = multi_output_regressor.score(test_input, test_target)
print(f'Model score: {score}')

predictions = multi_output_regressor.predict(test_input)

Model score: 0.5160619515913027
